In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import random
import torch
import torch.backends.cudnn as cudnn
from sklearn.neural_network import MLPRegressor

os.chdir('Resources/')

import warnings
warnings.filterwarnings("ignore")

In [2]:
def power(b, p, m):
    b %= m
    if p == 0:
        return 1
    j = power(b, p // 2, m)
    j = (j * j) % m
    if p % 2 == 1:
        j = (j * b) % m
    return j

def mod_inv(a, m):
    origin_m = m
    y, x = 0, 1
    if m == 1:
        return 0
    while a > 1:
        q = a // m
        t = m
        m = a % m
        a = t
        t = y
        y = x - q * y
        x = t
    if x < 0:
        x += origin_m
    return x

def encode_signed(m, p):
    return m + 9

def decode_signed(m_encoded, p):
    return m_encoded - (9 * 9)

def encrypt_additive(m, h, g, p, y=7):
    m_enc = encode_signed(m, p)
    c1 = power(g, y, p)
    s = power(h, y, p)
    c2 = (power(g, m_enc, p) * s) % p
    return c1, c2

def decrypt_additive(ciphertext, x, p, g):
    c1, c2 = ciphertext
    s = power(c1, x, p)
    s_inv = mod_inv(s, p)
    m_encoded = (c2 * s_inv) % p
    for m in range(p - 1):
        if power(g, m, p) == m_encoded:
            return decode_signed(m, p)
    return None

In [3]:
def reset_seeds(seed=42):
    import os
    import random
    import numpy as np
    import tensorflow as tf
    import torch
    import torch.backends.cudnn as cudnn

    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    torch.manual_seed(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False

def preprocess_dataset(df, seed=42):
    reset_seeds()

    X = df[['Plant_ID', 'Machine_Type', 'Quality_Audit', 'Year', 'Month', 'Week']].values
    y = df['Weekly_Production'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    
    return X, X_test, y, y_test


def build_mlp(input_dim):
    reset_seeds()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(100, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(50, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mse')
    return model

def train_local(X_train, X_test, y_train, y_test):
    reset_seeds()
    model = build_mlp(X_train.shape[1])
    model.fit(X_train, y_train, epochs=1000, batch_size=32, verbose=0)
    y_pred = model.predict(X_test)
    return model, r2_score(y_test, y_pred)

def get_gradients_and_flatten(model, X_train, y_train):
    mse_loss_fn = tf.keras.losses.MeanSquaredError()
    X_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
    y_tensor = tf.convert_to_tensor(y_train.reshape(-1, 1), dtype=tf.float32)
    
    with tf.GradientTape() as tape:
        predictions = model(X_tensor, training=True)
        loss = mse_loss_fn(y_tensor, predictions)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    flat_grads = []
    for g in gradients:
        flat = tf.reshape(g, [-1]).numpy()
        flat_grads.extend(flat)
    
    return flat_grads

def add_noise_to_gradients(flat_grads, noise_stddev=1.0):
    return [g + tf.random.normal(shape=g.shape, stddev=noise_stddev) for g in flat_grads]

def reconstruct_grads(flat_grads, model):
    reconstructed = []
    idx = 0
    for var in model.trainable_variables:
        shape = var.shape
        size = np.prod(shape)
        chunk = flat_grads[idx:idx + size]
        tensor = tf.convert_to_tensor(np.array(chunk, dtype=np.float32).reshape(shape))
        reconstructed.append(tensor)
        idx += size
    return reconstructed

def apply_gradients(model, avg_grads):
    reset_seeds()
    optimizer = tf.keras.optimizers.Adam()
    optimizer.apply_gradients(zip(avg_grads, model.trainable_variables))

def fine_tune(model, X_train, y_train):
    reset_seeds()
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)

In [4]:
p = 1009 # prime number
g = 11 # generator
x = 5 # private key
h = power(g, x, p) # public key (p, g, h)

clients = [f"2_{i}_Client_Data_" for i in range(1, 10)]
rounds = [
    "2010_2", "2010_3", "2010_4", "2010_5", "2010_6", "2010_7", "2010_8", "2010_9", "2010_10", "2010_11", "2010_12",
    "2011_1", "2011_2", "2011_3", "2011_4", "2011_5", "2011_6", "2011_7", "2011_8", "2011_9", "2011_10", "2011_11", "2011_12",
    "2012_1", "2012_2", "2012_3", "2012_4", "2012_5", "2012_6", "2012_7", "2012_8", 
    #"2012_9", "2012_10"
    ]

client_models = {}

client_r2_history = {i: [] for i in range(9)}

In [5]:
import time

start_time = time.time()

print("Round 2010_2")
train_data = {}
for idx, client in enumerate(clients):
    path = client + rounds[0] + ".csv"
    df = pd.read_csv(path)
    X_train, X_test, y_train, y_test = preprocess_dataset(df, seed=42)
    model, r2 = train_local(X_train, X_test, y_train, y_test)
    client_r2_history[idx].append(r2)
    print(f"R² of Client_{idx+1} ({rounds[0]}): {r2:.4f}")
    train_data[idx] = (X_train, X_test, y_train, y_test)
    client_models[idx] = model

Round 2010_2
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
R² of Client_1 (2010_2): 0.9999
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
R² of Client_2 (2010_2): 0.0200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
R² of Client_3 (2010_2): 0.9997
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
R² of Client_4 (2010_2): 0.9996
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
R² of Client_5 (2010_2): 0.7836
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
R² of Client_6 (2010_2): 0.9994
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
R² of Client_7 (2010_2): 0.9998
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
R² of Client_8 (2010_2): 0.9994
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
R² of Client_9 (2010_2): 0.9995


In [6]:
import time
from phe import paillier

# ------------------------------------------------------------------
# Small fixed Paillier key (sufficient for digit values [-81, 81])
# ------------------------------------------------------------------
p = 10007
q = 10009
n = p * q
public_key = paillier.PaillierPublicKey(n)
private_key = paillier.PaillierPrivateKey(public_key, p, q)

# ------------------------------------------------------------------
# Minimal signed-int helpers that BYPASS phe's EncodedNumber overflow
# ------------------------------------------------------------------
def encrypt_int(pk, m):
    m_mod = m % pk.n
    c = pk.raw_encrypt(m_mod)
    return paillier.EncryptedNumber(pk, c, 0)

def decrypt_int(sk, enc):
    m_mod = sk.raw_decrypt(enc.ciphertext())
    if m_mod > sk.public_key.n // 2:
        return m_mod - sk.public_key.n
    return m_mod

# ------------------------------------------------------------------
# Digit-splitting secure aggregation (no fixed length) with custom order
# ------------------------------------------------------------------
def paillier_digit_split_aggregate(client_models, train_data, scale=1_000_000):
    start_time = time.time()

    # ---- Collect & scale grads from each client ----
    scaled_grads_list = []
    all_split_grads = []
    max_digit_len = 20

    for i, (idx, model) in enumerate(client_models.items()):
        X_train, _, y_train, _ = train_data[idx]
        grad = get_gradients_and_flatten(model, X_train, y_train)
        grad = [int(round(x * scale)) for x in grad]
        print(f"Grad {i}:\n{grad}")
        scaled_grads_list.append(grad)

        split_grad = []
        for val in grad:
            digits = [int(d) for d in str(abs(val))]
            if val < 0:
                digits = [-d for d in digits]
            split_grad.append(digits)
        all_split_grads.append(split_grad)

    # ---- Organize digit splits from least to most significant ----
    digit_splits = []
    for pos in range(max_digit_len):
        pos_layer = []
        for client_grad in all_split_grads:
            layer = []
            for digits in client_grad:
                if len(digits) <= pos:
                    layer.append(None)
                else:
                    d = digits[-1 - pos]
                    layer.append(d)
            pos_layer.append(layer)
        digit_splits.append(pos_layer)

    # ---- Define a custom order for positions (example: a shuffled order) ----
    custom_order = [3, 1, 7, 0, 5, 2, 4, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

    print(f"Custom order for digit positions: {custom_order}")

    # ---- Encrypt & homomorphically add digit layers in custom order ----
    decrypted_digits_in_order = []
    total_enc_time = 0.0
    total_dec_time = 0.0

    for pos in custom_order:
        enc_clients = []

        enc_start = time.perf_counter()
        for client_digit_vec in digit_splits[pos]:
            enc_vec = []
            for x in client_digit_vec:
                if x is None:
                    enc_vec.append(None)
                else:
                    enc_vec.append(encrypt_int(public_key, x))
            enc_clients.append(enc_vec)
        enc_end = time.perf_counter()
        total_enc_time += (enc_end - enc_start)

        enc_sum_vec = []
        for col_i in range(len(enc_clients[0])):
            acc = None
            for c_idx in range(len(enc_clients)):
                val = enc_clients[c_idx][col_i]
                if val is not None:
                    if acc is None:
                        acc = val
                    else:
                        acc += val
            enc_sum_vec.append(acc)

        dec_start = time.perf_counter()
        dec_vec = [decrypt_int(private_key, c) if c is not None else 0 for c in enc_sum_vec]
        dec_end = time.perf_counter()
        total_dec_time += (dec_end - dec_start)

        decrypted_digits_in_order.append(dec_vec)

    # ---- Reorder decrypted digits back to natural position before reconstruction ----
    reordered_decrypted_digits = [None] * max_digit_len
    for idx, pos in enumerate(custom_order):
        reordered_decrypted_digits[pos] = decrypted_digits_in_order[idx]

    print(f"Reordered digit positions after decryption: {[i for i in range(max_digit_len)]}")

    print(f"Total encryption time: {total_enc_time:.4f} seconds")
    print(f"Total decryption time: {total_dec_time:.4f} seconds")

    # ---- Reconstruct summed gradients from digit columns ----
    start_reconstruct = time.perf_counter()
    grad_sums_scaled = []
    for i in range(len(reordered_decrypted_digits[0])):
        val = 0
        for pos in range(len(reordered_decrypted_digits)):
            digit = reordered_decrypted_digits[pos][i]
            val += digit * (10 ** pos)  # least to most significant
        grad_sums_scaled.append(val)
    reconstruct_time = time.perf_counter() - start_reconstruct
    print(f"Total reconstruction time: {reconstruct_time:.4f} seconds")

    grad_sums = [v / scale for v in grad_sums_scaled]
    print("Sum of Gradients:", grad_sums)

    num_clients = len(scaled_grads_list)
    grads_avg = [round(v / num_clients, 6) for v in grad_sums]
    print("Average of Gradients:", grads_avg)

    end_time = time.time()
    print(f"Total time: {end_time - start_time:.2f} seconds")

    return grad_sums, grads_avg, digit_splits, reordered_decrypted_digits

# ------------------------------------------------------------------
# Run
# ------------------------------------------------------------------
grad_sums, grads_avg, digit_splits, decrypted_digits = paillier_digit_split_aggregate(
    client_models,
    train_data,
    scale=1_000_000,
)

Grad 0:
[-124043213, -12660294, -346210419, 0, 0, 0, -227913498, 199307602, -344809387, 0, -342722595, -350888733, -115159660, -9506560547, -565449707, -1623532, 211443100, -482686188, 0, 0, 0, 199359772, 0, 0, 0, 0, 0, -194833679, 160492157, 0, 219608414, 0, 0, 0, 0, 343291901, 0, 0, 0, -182670547, -496260406, 0, 0, 160821945, 0, -11707053711, 0, -4994146484, 0, -600630798, 0, 0, -5672741, 0, -5627257324, 0, 0, -186006485, 0, 0, 0, 258376129, 611496826, 0, 0, 105977020, 0, 330403442, 0, 0, 0, 0, 309689941, -448555542, 0, 0, 0, 0, 0, 225309875, 0, 334858826, 0, 473792419, -9771427734, -6460286133, 0, 0, 305209503, 430681213, 0, 0, -146387924, 0, 0, 148696991, -179765472, -5943419434, 291036804, 0, 4883692871, 683571655, 13549728516, 0, 0, 0, 8839670898, -7775488281, 13519633789, 0, 13377932617, 13658075195, 4486717773, -257281312500, 22043632812, 31583099, -8173721191, 18851343750, 0, 0, 0, -7791199219, 0, 0, 0, 0, 0, 7524835938, -6254746582, 0, -8546186523, 0, 0, 0, 0, -13380316406, 0

In [7]:
def estimate_communication_size(digit_splits, summed_ciphertexts, public_key):
    """
    Compute size of encrypted data sent from each client to the server,
    and the size of the aggregated encrypted data sent back to clients.
    """
    bits_per_ciphertext = public_key.n.bit_length()

    num_clients = len(digit_splits[0])  # list of clients per digit position
    num_digits = len(digit_splits)
    num_features = len(digit_splits[0][0])  # length of gradient vector

    # From each client to server
    for client_idx in range(num_clients):
        total_ciphertexts = 0
        for pos in range(num_digits):
            total_ciphertexts += len(digit_splits[pos][client_idx])
        total_bits = total_ciphertexts * bits_per_ciphertext
        print(f"Total size from Client {client_idx} to Server: {total_bits} bits")

    # From server to clients
    total_sum_ciphertexts = num_digits * len(summed_ciphertexts[0])  # 1 sum per digit layer
    total_server_to_client_bits = total_sum_ciphertexts * bits_per_ciphertext
    print(f"Total size from Server to Clients (summed ciphertexts): {total_server_to_client_bits} bits")

# For data size reporting
estimate_communication_size(digit_splits, decrypted_digits, public_key)


Total size from Client 0 to Server: 3132540 bits
Total size from Client 1 to Server: 3132540 bits
Total size from Client 2 to Server: 3132540 bits
Total size from Client 3 to Server: 3132540 bits
Total size from Client 4 to Server: 3132540 bits
Total size from Client 5 to Server: 3132540 bits
Total size from Client 6 to Server: 3132540 bits
Total size from Client 7 to Server: 3132540 bits
Total size from Client 8 to Server: 3132540 bits
Total size from Server to Clients (summed ciphertexts): 3132540 bits


In [ ]:
import time
import pandas as pd

start_time = time.time()

for round_id in rounds[1:]:
    print(f"\nRound ({round_id})")
    new_train_data = {}
    new_client_models = {}
    round_index = rounds.index(round_id)

    for idx, client in enumerate(clients):
        # Merge data from previous and current rounds
        dfs = []
        for r in rounds[:round_index + 1]:
            path = f"{client}{r}.csv"
            df_ind = pd.read_csv(path)
            dfs.append(df_ind)
        df = pd.concat(dfs, ignore_index=True)

        # Preprocess data
        X_train, X_test, y_train, y_test = preprocess_dataset(df, seed=42)

        # Build and apply previous averaged gradient
        model = build_mlp(X_train.shape[1])
        model(X_train[:1])  # initialize model weights
        avg_grads_tensor = reconstruct_grads(grads_avg, model)
        apply_gradients(model, avg_grads_tensor)

        # Fine-tune using client data
        fine_tune(model, X_train, y_train)

        # Evaluate and store
        r2 = r2_score(y_test, model.predict(X_test))
        client_r2_history[idx].append(r2)
        print(f"R² of Client_{idx+1} ({round_id}): {r2:.4f}")

        # Store updated model and data
        new_train_data[idx] = (X_train, X_test, y_train, y_test)
        new_client_models[idx] = model

    # --- Perform Digit-Splitting Secure Aggregation ---
    grad_sums, grads_avg, digit_splits, decrypted_digits = paillier_digit_split_aggregate(
        new_client_models,
        new_train_data,
        scale=1_000_000,
    )

    # --- Update for next round ---
    train_data = new_train_data
    client_models = new_client_models

end_time = time.time()
print(f"\nSecure Digit-Splitting FL completed in {(end_time - start_time):.2f} seconds")


Round (2010_3)
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
R² of Client_1 (2010_3): 0.1490
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
R² of Client_2 (2010_3): 0.0025
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
R² of Client_3 (2010_3): 0.1523
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
R² of Client_4 (2010_3): 0.0657
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
R² of Client_5 (2010_3): -0.2142
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
R² of Client_6 (2010_3): 0.2250
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
R² of Client_7 (2010_3): 0.2081
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
R² of Client_8 (2010_3): 0.2590
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
R² of Client_9 (2010_3): 0.2145
Grad 0:
[27305750000, 36820710938, 26180857422, 0, 0, 0, 921665771, 5001452148, 29721117188, 0, 15613258789, 6004440918, 11755226562, 44887898438, 12140753906, 1768531738, 21809470703, 21912527344, 0, 33992941406, 0, 3356973877, 0, 0, 0, 0, 0, -4203144531, 7630014648, 0, 11217348633, 0, 0, 0, 0, 5443384766, 0, 0, 0, 3969336426, 3024789551, 0,